# Exercises week 43 and 44
### The OR, AND, and XOR gates

We have two input values $x_1$ and $x_2$ which decide the output from the two types of gates. Since each input value can be either 0 or 1 we can write the input as a design matrix $X$ where the first and second column represents $x_1$ and $x_2$ respectively as:
$$X = \begin{bmatrix} 0 & 0 \\ 0 & 1 \\ 1 & 0 \\ 1 & 1 \end{bmatrix}$$

The output $y$ for the different gates we can write as the vectors $y^T=[0, 1,1,1]$ for the OR gate, $y^T=[0,0,0,1]$ for the AND gate, and $y^T=[0, 1, 1, 0]$ for the XOR gate. We setup this matrix and these vectors:


In [113]:
import numpy as np
import jax.numpy as jnp
from jax import grad

# Set up design matrix and output vectors
X = np.asarray([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# Gate target arrays
yOR = np.asarray([0, 1, 1, 1]).reshape(-1, 1)
yAND = np.asarray([0, 0, 0, 1]).reshape(-1, 1)
yXOR = np.asarray([0, 1, 1, 0]).reshape(-1, 1)

We create our NN architecture with the Sigmoid function $\sigma$ as activation function where
\begin{equation}
    \sigma(x) = \frac{1}{1+e^{-x}}
\end{equation}
as such

In [114]:
# Parameters
n_hidden_nodes = 2  # hidden nodes per layer
n_categories = 1  # output nodes
n_inputs, n_features = X.shape  # 2 inputs, 4 features
target_gate = "OR"  # choose which target gate to train on


# Activation function 
def activation(x):
    # Use sigmoid function:
    return 1 / (1 + np.exp(-x))


# Initialize random number generator with seed
rng = np.random.default_rng(2023)

# Weights and bias in the hidden layer
hidden_weights = rng.standard_normal((n_features, n_hidden_nodes))  # weights normally distributed
hidden_bias = np.zeros(n_hidden_nodes) + 0.01

# Weights and bias in the output layer
output_weights = rng.standard_normal((n_hidden_nodes, n_categories))  # weights normally distributed
output_bias = np.zeros(n_categories) + 0.01


print("Before back propagation:\n")
print("Output weights and bias:")
print(output_weights)
print(output_bias)

print()
print("Hidden weights and bias:")
print(hidden_weights)
print(hidden_bias)

Before back propagation:

Output weights and bias:
[[-0.77586755]
 [ 0.8087058 ]]
[0.01]

Hidden weights and bias:
[[ 0.60172129  1.15161897]
 [-1.35946236  0.22205533]]
[0.01 0.01]


### Feed forward
Then we set up the feed forward algorithm and compare one pass with the target vectors $y^T$

In [115]:
# Get the chosen target array
if target_gate == "OR":
    target = yOR
elif target_gate == "AND":
    target = yAND
elif target_gate == "XOR":
    target = yXOR
else:
    raise ValueError(f"Target gate not found, got '{target_gate}'.")

In [128]:
def feed_forward(X):
    """Feed forward algorithm for one hidden layer"""
    # Weighted sum of inputs to the hidden layer
    z_h = X @ hidden_weights + hidden_bias

    # Activation in the hidden layer
    a_h = activation(z_h)

    # Weighted sum of inputs to the output layer
    z_o = a_h @ output_weights + output_bias

    # Activation of output layer; contains the output probabilities
    probabilities = activation(z_o)  # this is a_o
    return a_h, probabilities


@np.vectorize
def predict(probability):
    """Get prediction from array with floats. Step function."""
    if not isinstance(probability, (int, float)):
        raise ValueError(f"Probability must be a number, int or float. Got {type(probability)}")
    if probability < 0.5:
        return 0
    else:
        return 1


# Make prediction and compare with gate target y_vectors
a_h, probabilities = feed_forward(X)
predictions = predict(probabilities)

print("Target:")
print(target)

print("\nProbabilities:")
print(probabilities)

print("\nPrediction:")
print(predictions)

True
Target:
[[0]
 [1]
 [1]
 [1]]

Probabilities:
[[0.00136189]
 [0.99940278]
 [0.99939456]
 [0.99986535]]

Prediction:
[[0]
 [1]
 [1]
 [1]]


We see this prediction does not match any target. This is because we only did one pass and that was with random starting weights. Now we setup the cost function and the back propagation algorithm.

<!--- 
For the cost function we use the cross entropy for binary classification given as
$$C(\boldsymbol{\theta}) = -\sum_{i=1}^n \left( y_i \ln [p(y_i | x_i, \boldsymbol{\theta)}] + (1-y_i)\ln[1-p(y_i | x_i, \boldsymbol{\theta)}] \right)$$

where the probabilities $p$ we have from the sigmoid function 
$$p(y_i=1|x_i, \boldsymbol{\theta}) = \frac{\exp(\theta_0 + \theta_1 x_i)}{1- \exp(\theta_0 + \theta_1 x_i)}$$
$$p(y_i=0|x_i, \boldsymbol{\theta}) = 1 - p(y_i=1|x_i, \boldsymbol{\theta})$$ 
-->

For the cost function we use the cross entropy for binary classification given as
$$C(\boldsymbol{W}) = -\sum_{i=1}^n \left( t_i \log a_i^L + (1-t_i)\log(1-a_i^L) \right)$$
where $t$ is the target and $a^L$ is the final activation from the final/output layer.

In [119]:
def cost_log_reg(target):
    """Returns a function for the logistic cross entropy for binary classification / log loss function using a given target vector."""
    d = 1e-9  # small value to avoid infinities

    def func(x):
        #     return -(1 / target.size) * jnp.sum(target * jnp.log(x + d))
        #     return -np.sum(target * jnp.log(x + d) + (1 - target) * jnp.log(1 - x + d))
        return -jnp.sum(
                (target * jnp.log(x + d)) + ((1 - target) * jnp.log(1 - x + d))
        )

    return func


cost_func = cost_log_reg(target)
cost_func(predictions)

Array(20.723267, dtype=float32)

### Calculating the analytical gradients for back propagation

[BEGIN OVER AGAIN WITH THIS: START WITH W41 (Bringing it together, first back propagation equation)]

The gradients are
\begin{equation}
    \nabla W_L=a_{L-1}^T \delta_L  = a_{h}^T \delta_L 
\end{equation}
\begin{equation}
\end{equation}
but for our only hidden layer we have
\begin{equation}
    \nabla W_h = X^T\delta_h
\end{equation}
and the gradients for the biases will be
\begin{align}
    \nabla b_L &= \sum_i^{n_{inputs}} \delta_L
    \\ \nabla b_h &= \sum_i^{n_{inputs}} \delta_h
\end{align}

The expression we need to now calculate is  the expression for the output error $\delta_L$:

We differentiate the cost function with regards to the activation of the output layer $a_L$ and get:
\begin{align*}
    \frac{\partial C}{\partial a_L} &= -\frac{\partial}{\partial a_L}(t \ln(a_L) + (1-t)\ln(1-a_L))
    \\ &= -(\frac{t_}{a_L} + \frac{1-t_}{1-a_L}(-1))
    \\ &= \frac{1-t_}{1-a_L} - \frac{t_}{a_L}
    \\ &= \frac{a_L(1-t_)}{a_L(1-a_L)} - \frac{t_(1-a_L)}{a_L(1-a_L)}
    \\ &= \underline{\frac{a_L-t_}{a_L(1-a_L)}}
\end{align*}

The expression for the output error $\delta_L$ is 

\begin{equation}
    \delta_i^L = \sigma'(z_i^L)\frac{\partial C}{\partial a_i^L}
\end{equation}



where $\sigma$ is our Sigmoid function, we can differentiate it to be

\begin{equation}
    \sigma'(x) = \frac{e^{-x}}{(1+e^{-x})^2}
\end{equation}
which gives the following when putting in $a_i^L=\sigma(a_i^L)=1/(1+e^{-z_i^L})$:
\begin{gather*}
    \sigma'(z_i^L) &= \frac{e^{-z_i^L}}{(1+e^{-z_i^L})^2}
    \\ &= \left( \frac{1}{a_i^L} - 1 \right) (a_i^L)^2
    \\ &= \underline{a_i^L(1-a_i^L)}
\end{gather*}

which gives 
\begin{align*}
    \delta_L &= a_L(1-a_L)\ \frac{a_L-t}{a_L(1-a_L)}
    \\ &= \underline{a_L-t}
\end{align*}



In [120]:
def analytic_error(target):
    """Analytical expression for output layer error"""

    def func(a):
        return a - target

    return func


analy_grad = analytic_error(target)

### Using automatic differentiation for the gradient

In [122]:
def dfdx(a):
    """Derivative of the activation function;
    rewritten expression of the derivative for sigmoid function"""
    return a * (1 - a)


def automatic_error(target):
    """Automatic differentiation output layer error"""
    dCda_L = grad(cost_log_reg(target))

    def func(a):
        return dfdx(a) * dCda_L(a)

    return func


auto_grad = automatic_error(target)

# Compare analytical vs automatic with target predictions
print(analy_grad(probabilities))
print()
print(auto_grad(probabilities))

[[ 0.50662492]
 [-0.4252487 ]
 [-0.46931083]
 [-0.39955287]]

[[ 0.50662494]
 [-0.42524868]
 [-0.46931082]
 [-0.39955288]]


### Back propagation
The results are the same, so I use the analytical expression since it is faster than using autograd each run.

Since we only have one hidden layer we need to propogate only once from the output layer to the first hidden layer

In [123]:
# Choose gradient method
error_func = analy_grad  # [analy_grad / auto_grad]


def back_propagate(X):
    """Back propagation algorithm for one hidden layer"""
    a_h, probabilities = feed_forward(X)

    # Output layer error delta^L
    error_o = error_func(probabilities)

    # Hidden layer error delta^1
    error_h = error_o @ output_weights.T * dfdx(a_h)

    # Gradients for the hidden layer
    hidden_weights_gradient = X.T @ error_h
    hidden_bias_gradient = np.sum(error_h, axis=0)

    # Gradients for the output layer
    output_weights_gradient = a_h.T @ error_o
    output_bias_gradient = np.sum(error_o, axis=0)

    return hidden_weights_gradient, hidden_bias_gradient, output_weights_gradient, output_bias_gradient

### Iterating until convergence

In [124]:
eta = 0.9  # learning rate
tol = 1e-7  # convergence check tolerance 
n_epochs = 1000  # max number of epochs 

for i in range(n_epochs):
    # Feed forward and back propagate back to get all gradients for each epoch
    dWh, dBh, dWo, dBo = back_propagate(X)
    
    # Update weights and biases
    output_weights -= eta * dWo
    output_bias -= eta * dBo
    hidden_weights -= eta * dWh
    hidden_bias -= eta * dBh

print("After back propagation:")
print("Output weights and bias:")
print(output_weights)
print(output_bias)

print()
print("Hidden weights and bias:")
print(hidden_weights)
print(hidden_bias)

print("\nTarget")
print(target)
print("\nProbabilities:")
probabilities = feed_forward(X)[-1]
print(probabilities)
print("\nPrediction")
print(predict(probabilities))

After back propagation:
Output weights and bias:
[[-7.74245141]
 [ 9.48880322]]
[-0.55654992]

Hidden weights and bias:
[[-4.06189703  5.02734516]
 [-4.26100841  4.8273186 ]]
[ 2.01086756 -2.40256552]

Target
[[0]
 [1]
 [1]
 [1]]

Probabilities:
[[0.00136189]
 [0.99940278]
 [0.99939456]
 [0.99986535]]

Prediction
[[0]
 [1]
 [1]
 [1]]


We get the correct prediction for both the OR and the AND gates, however the XOR gate gets predicted uncorrectly.

[PLOT ACCURACY AS FUNCTION OF LEARNING RATE]